In [1]:
import preprocessing as pp
import sto_mod as sm
import pandas as pd
import numpy as np
import time 
import xpress as xp # FICO Xprerss Solver
xp.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')

In [ ]:
# Read and Preprocess the data
customer_df, candidate_df, supplier_df, vehicle_df, distance_w_to_s_df, distance_w_to_c_df, demand_cus_period_df, demand_cus_period_scene_df = pp.read_and_prep_data()

# Customer Clustering
customer_df, cluster_center_df = pp.const_cluster_by_cus_loc(customer_df, n_clusters=50, size_min=4, size_max=40, random_state=42)

agg_dem_cus_period_scene_df = pp.agg_dem_cus_period_scene(demand_cus_period_scene_df, customer_df)

distance_w_to_cluster_df = pp.create_dis_mat_df(candidate_df, cluster_center_df,'cityblock')

# Create Cost
cost_w_to_cluster = pp.calculate_cost_from_w_to_cluster(distance_w_to_cluster_df, vehicle_df)
cost_w_to_s = pp.calculate_cost_from_w_to_s(distance_w_to_s_df, vehicle_df, supplier_df)


In [ ]:
n_clus_scene = 3
time_limit_s = 3600
time_obj_gap_dict = {}
n_clusters_scene_list = [3]

df,scene_cluster_center_df = pp.constrained_kmeans_clustering(agg_dem_cus_period_scene_df, n_clusters=n_clus_scene, 
                                                                size_min=np.floor(len(agg_dem_cus_period_scene_df.columns)/n_clus_scene), 
                                                                size_max=len(agg_dem_cus_period_scene_df.columns), random_state=42)
agg_dem_cus_period_clus_scene_df = pp.agg_scene_df(agg_dem_cus_period_scene_df, scene_cluster_center_df)

solve_time, obj_value, mip_gap_percent, x_matrix, y_matrix, v_matrix, z_matrix = sm.sto_model(  agg_dem_cus_period_clus_scene_df,
                                                                                                candidate_df,
                                                                                                supplier_df,
                                                                                                cluster_center_df,
                                                                                                cost_w_to_s,
                                                                                                cost_w_to_cluster,
                                                                                                time_limit_s)

/Users/thee/miniconda/envs/trm_conda/lib/python3.11/site-packages/k_means_constrained/k_means_constrained_.py:464: RuntimeWarning: invalid value encountered in cast
  costs = np.around(costs * 1000, 0).astype('int32')  # Times by 1000 to give extra precision


FICO Xpress v9.3.5, Hyper, solve started 18:53:55, Feb 13, 2025
Heap usage: 1864MB (peak 1864MB, 620MB system)
Minimizing MILP MEWLP_Stochastics using up to 11 threads and up to 18GB memory, with these control settings:
MAXTIME = 3600
OUTPUTLOG = 1
MIPRELSTOP = .001
Original problem has:
   3376390 rows      3538040 cols     13502720 elements    664840 entities
Presolved problem has:
    722550 rows       887206 cols      6150987 elements    654006 entities
LP relaxation tightened
Presolve finished in 37 seconds
Heap usage: 2618MB (peak 4498MB, 620MB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  8.00e+06] / [ 8.52e-03,  1.31e+02]
  RHS and bounds [min,max] : [ 1.00e+00,  1.53e+07] / [ 1.00e+00,  9.36e+02]
  Objective      [min,max] : [ 1.73e-04,  6.25e+06] / [ 2.51e-01,  6.88e+06]
Autoscaling applied Curtis-Reid scaling

Will try to keep branch and bound tree memory usage below 9.4GB
Starting concurrent s

In [ ]:
agg_dem_cus_period_clus_scene_df

0         1         2         3   \
Cluster ProductIndex PeriodIndex                                           
0       0            0             6072.00   6464.25   5913.75   5921.50   
                     1            11935.25  12071.00  12163.50  11140.25   
                     2            18221.00  19384.75  19938.75  15609.50   
                     3            25393.50  27024.75  25676.00  20619.25   
                     4            26681.75  32079.50  29329.75  23825.50   
...                                    ...       ...       ...       ...   
99      3            5              136.00    159.00    154.50    108.50   
                     6              164.00    207.25    184.75    139.25   
                     7              174.00    211.00    193.75    140.75   
                     8              192.00    297.50    232.00    154.75   
                     9              205.50    294.00    235.25    146.50   

                                        4         5         6         7   \
Cluster ProductIndex PeriodIndex                                           
0       0            0             6198.00   5930.00   6015.50   6169.50   
                     1            12911.00  12318.25  11528.50  12079.25   
                     2            19897.75  16827.50  16202.25  17530.50   
                     3            28769.00  23286.25  20656.25  24954.25   
                     4            35642.50  26436.25  22490.00  28472.75   
...                                    ...       ...       ...       ...   
99      3            5              184.00    132.50    119.50    140.25   
                     6              249.75    162.25    138.00    173.00   
                     7              241.00    170.50    127.25    199.75   
                     8              288.50    209.00    136.00    213.75   
                     9              340.25    213.75    139.50    239.25   

                                        8         9   ...        15        16  \
Cluster ProductIndex PeriodIndex                      ...                       
0       0            0             6265.25   6466.25  ...   6101.00   6242.00   
                     1            13772.25  13274.75  ...  12494.75  11952.75   
                     2            21811.75  20816.25  ...  19150.75  18240.50   
                     3            30332.25  28868.25  ...  26168.00  26242.50   
                     4            37058.25  37321.75  ...  31646.25  29243.50   
...                                    ...       ...  ...       ...       ...   
99      3            5              220.00    192.50  ...    155.50    152.50   
                     6              274.75    242.50  ...    201.25    197.50   
                     7              314.00    294.75  ...    227.00    200.75   
                     8              359.00    359.75  ...    246.00    238.75   
                     9              399.25    373.00  ...    283.00    270.25   

                                        17        18        19        20  \
Cluster ProductIndex PeriodIndex                                           
0       0            0             6001.50   6403.75   6684.50   5820.25   
                     1            11757.50  14449.25  13590.00  13445.50   
                     2            17180.25  21254.25  21062.25  20103.50   
                     3            24471.25  32092.50  28936.75  29336.00   
                     4            27189.50  39924.75  33242.25  34478.25   
...                                    ...       ...       ...       ...   
99      3            5              139.50    207.00    157.50    177.25   
                     6              160.25    284.00    234.00    221.00   
                     7              182.00    298.50    255.75    250.00   
                     8              189.75    386.75    285.75    297.00   
                     9              212.25    400.25    328.75    337.25   

                   

In [ ]:
# time_limit_s = 3600

# model = xp.problem(name= 'MEWLP_Stochastics')
# # Sets and Notation
# S = list(supplier_df.index) # Supplier Index
# S_P0 = list(supplier_df[supplier_df['SupplierProductGroup'] == 0]['SupplierProductGroup'].index)
# S_P1 = list(supplier_df[supplier_df['SupplierProductGroup'] == 1]['SupplierProductGroup'].index)
# S_P2 = list(supplier_df[supplier_df['SupplierProductGroup'] == 2]['SupplierProductGroup'].index)
# S_P3 = list(supplier_df[supplier_df['SupplierProductGroup'] == 3]['SupplierProductGroup'].index)
# S_P_dict = {0:S_P0, 1:S_P1, 2:S_P2, 3:S_P3}

# W = list(candidate_df.index) # Warehouse Index
# C = list(cluster_center_df.index) # Cluster Index
# P = list(agg_dem_cus_period_clus_scene_df.reset_index()['ProductIndex'].unique())
# T = list(agg_dem_cus_period_clus_scene_df.reset_index()['PeriodIndex'].unique())
# Phi = list(agg_dem_cus_period_clus_scene_df.columns)

# # Output Variables
# x = np.array([xp.var(f'x_{w}_{c}_{t}_{xi}', vartype = xp.binary) for w in W for c in C for t in T for xi in Phi], dtype = xp.npvar).reshape(len(W), len(C), len(T), len(Phi))
# y = np.array([xp.var(f'y_{w}_{t}', vartype = xp.binary) for w in W for t in T], dtype = xp.npvar).reshape(len(W), len(T))
# o = np.array([xp.var(f'o_{w}', vartype = xp.binary) for w in W], dtype = xp.npvar).reshape(len(W))
# v = np.array([xp.var(f'v_{w}_{c}_{p}_{t}_{xi}', vartype = xp.continuous, lb = 0) for w in W for c in C for p in P for t in T for xi in Phi], dtype = xp.npvar).reshape(len(W), len(C), len(P), len(T), len(Phi))
# z = np.array([xp.var(f'z_{w}_{s}_{t}', vartype = xp.continuous, lb = 0) for w in W for s in S for t in T], dtype = xp.npvar).reshape(len(W), len(S), len(T))
# # a = np.array([xp.var(f'a_{w}_{c}_{p}_{t}_{xi}', vartype = xp.continuous, lb = 0) for w in W for c in C for p in P for t in T for xi in Phi], dtype = xp.npvar).reshape(len(W),len(C), len(P), len(T), len(Phi))
# # b = np.array([xp.var(f'b_{w}_{p}_{t}_{xi}', vartype = xp.continuous, lb = 0) for w in W for p in P for t in T for xi in Phi], dtype = xp.npvar).reshape(len(W), len(P), len(T), len(Phi))

# # model.addVariable(x, y, o, v, z, a, b)
# model.addVariable(x, y, o, v, z)


# # Constraints 
# for w in W:
#     for t in T:
#         model.addConstraint(xp.constraint(o[w] >= y[w,t]))
#         if t != 0:
#             model.addConstraint(xp.constraint(y[w, t] >= y[w,t-1]))    

#         for c in C:
#             for xi in Phi:
#                 model.addConstraint(xp.constraint(x[w,c,t,xi] <= y[w,t]))  

# for xi in Phi:
#     for t in T:
#         for w in W:
#             Capacity_W = candidate_df.loc[w,'Capacity']
#             model.addConstraint(xp.constraint(xp.Sum(v[w,c,p,t,xi] for c in C for p in P) <= Capacity_W * y[w,t]))
#         for c in C:
#             model.addConstraint(xp.constraint(xp.Sum(x[w,c,t,xi] for w in W) == 1))

# for xi in Phi:
#     for t in T:
#         for c in C:
#             for p in P:
#                 Demand = agg_dem_cus_period_scene_df.loc[(c,p,t),xi]
#                 for w in W:
#                     # model.addConstraint(xp.constraint(v[w,c,p,t,xi] + a[w,c,p,t,xi] == Demand * x[w,c,t,xi]))
#                     model.addConstraint(xp.constraint(v[w,c,p,t,xi] >= Demand * x[w,c,t,xi]))

# for xi in Phi:
#     for t in T:
#         for w in W:
#             for p in P:
#                 # model.addConstraint(xp.constraint(xp.Sum(z[w,s,t] for s in S_P_dict[p]) + b[w,p,t,xi] >= xp.Sum(v[w,c,p,t,xi] for c in C)))
#                 model.addConstraint(xp.constraint(xp.Sum(z[w,s,t] for s in S_P_dict[p]) >= xp.Sum(v[w,c,p,t,xi] for c in C)))
                
# for s in S:
#     Capacity_S = supplier_df.loc[s,'SupplierCapacity']
#     for t in T:
#         model.addConstraint(xp.constraint(xp.Sum(z[w,s,t] for w in W) <= Capacity_S))

# Setup_cost = xp.Sum(candidate_df.loc[w,'Setup'] * o[w] for w in W)
# Operating_cost = xp.Sum(candidate_df.loc[w,'Operating'] * y[w,t] for w in W for t in T)
# Tra_w_s_cost = xp.Sum(cost_w_to_s.loc[w,s] * z[w,s,t] for w in W for s in S for t in T)

# equal_prob = 1/len(agg_dem_cus_period_clus_scene_df.columns)
# # ld = 10000000000
# # gm = 10000000000/2
# Recourse = 0

# for xi in Phi:
#     Tra_w_c_cost = xp.Sum(cost_w_to_cluster.loc[w,c] * v[w,c,p,t,xi] for w in W for c in C for p in P for t in T)
#     # Pen_w_c = ld* xp.Sum(a[w,c,p,t,xi] for w in W for c in C for p in P for t in T)
#     # Pen_w_s = gm* xp.Sum(b[w,p,t,xi] for w in W for p in P for t in T)
#     # Recourse += equal_prob * (Tra_w_c_cost + Pen_w_c + Pen_w_s)
#     Recourse += equal_prob * (Tra_w_c_cost)

# obj = Setup_cost + Operating_cost + Tra_w_s_cost + Recourse
# # model.addObjective(Setup_cost, Operating_cost, Tra_w_s_cost, Recourse)
# model.setObjective(obj, sense = xp.minimize)

# model.setControl('miprelstop', 1e-3)
# model.setControl('maxtime', time_limit_s)
# tic_time = time.time()
# # Solve the problem
# model.solve()
# toc_time = time.time()
# solve_time = toc_time - tic_time
# obj_value = model.getObjVal()

# mip_gap_percent = 100*(obj_value - model.getAttrib('bestbound'))/obj_value
# print(f'Solving Time: {solve_time}')
# print(f'Objective Value: {obj_value}')
# print(f'%Gaps: {mip_gap_percent}')


KeyboardInterrupt: 

In [ ]:
obj_value

49376577.77833026

In [ ]:
time_obj_gap_dict = {}
time_obj_gap_dict[n_clus_scene] = [solve_time, obj_value, mip_gap_percent]

In [ ]:
asdfasdfasdfasdf

In [ ]:
time_obj_gap_dict

{3: [3600.946367263794, 49376577.77833026, 3.022741862847515]}

{3: [3600.946367263794, 49376577.77833026, 3.022741862847515]}